In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
import datasets
import numpy as np

pl.seed_everything(89026614)

text = datasets.load_dataset('tiny_shakespeare')["train"][0]["text"]
#text = open('../../Downloads/simplebooks/simplebooks-2-raw/train.txt', 'r').read()

Global seed set to 89026614
Found cached dataset tiny_shakespeare (/Users/cztomsik/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 828.10it/s]


In [5]:
device = "cpu"
block_size = 64

class MyDataset(data.Dataset):
    def __init__(self, text):
        super().__init__()
        vocab = sorted(set(text))
        self.vocab = vocab
        self.stoi = { ch: i for i, ch in enumerate(vocab) }
        self.itos = { i: ch for i, ch in enumerate(vocab) }
        self.data = torch.tensor([self.stoi[ch] for ch in text], dtype=torch.long).to(device)

    def __len__(self):
        return self.data.size(0) - block_size - 1

    def __getitem__(self, i):
        end = i + block_size
        return self.data[i:end], self.data[i + 1:end + 1]

dataset = MyDataset(text)
train = data.Subset(dataset, range(0, len(dataset) - 500))
test = data.Subset(dataset, range(len(dataset) - 500, len(dataset)))

In [6]:
# inspired by https://github.com/karpathy/minGPT/blob/master/mingpt/model.py
class MinGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, dropout):
        super().__init__()
        self.transformer = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.Dropout(dropout),
            nn.Sequential(*[DecoderLayer(embed_dim, dropout) for _ in range(num_layers)]),
            nn.LayerNorm(embed_dim),
        )
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, x):
        return self.lm_head(self.transformer(x))

    @torch.no_grad()
    def generate(self, input_ids, max_new_tokens, top_k=10):
        for _ in range(max_new_tokens):
            out = self(input_ids[:, -block_size:])
            logits = out[:, -1, :]
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("Inf")
            step_res = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            # auto-regression
            input_ids = torch.cat((input_ids, step_res), dim=1)

        return input_ids

class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = AFTFullCumsum(embed_dim, dropout)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(approximate="tanh"),
            nn.Linear(4 * embed_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.register_buffer("mask", ~torch.tril(torch.ones(block_size, block_size)).to(bool).to(device))

    def forward(self, x):
        B, T, _ = x.shape
        x = self.ln1(x)
        x = x + self.attn(x, self.mask)
        x = x + self.mlp(self.ln2(x))
        return x

# https://arxiv.org/pdf/2105.14103.pdf
# but w is not exponentiated and we divide by cumulative sum of exp(k) instead of exp(w) @ exp(k)
class AFTFullCumsum(nn.Module):
    def __init__(self, embed_dim, dropout):
        super().__init__()
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.wbias = nn.Parameter(torch.ones(block_size, block_size), requires_grad=True)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.drop = nn.Dropout(dropout)
    
    def forward(self, x, mask):
        B, T, C = x.shape
        q, k, v = self.qkv(x).chunk(3, dim=-1)

        w = self.wbias[:T,:T].masked_fill(mask[:T, :T], 0)
        exp_k = k.exp()

        weighted_avg = torch.einsum("ik,bkj->bij", w, exp_k * v) / torch.cumsum(exp_k, dim=1)
        Yt = F.sigmoid(q) * weighted_avg

        return self.drop(self.proj(Yt))


class Model(pl.LightningModule):
    def __init__(self, vocab_size, lr=0.004):
        super().__init__()
        self.lr = lr
        self.model = MinGPT(vocab_size, embed_dim=192, num_layers=3, dropout=0.1)
    
    def forward(self, x, y=None):
        logits = self.model(x)
        return logits if y is None else F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), ignore_index=-1)

    def train_dataloader(self):
        return data.DataLoader(train, batch_size=64, num_workers=0, sampler=data.RandomSampler(train, False, 5_000))

    def training_step(self, batch, batch_idx):
        return self(*batch)

    def val_dataloader(self):
        return data.DataLoader(test, batch_size=16, num_workers=0)
    
    def validation_step(self, batch, batch_idx):
        loss = self(*batch)
        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_epoch_end(self, outs):
        with torch.no_grad():
            x = torch.tensor([dataset.stoi[s] for s in "And now "], dtype=torch.long)[None,...].to(device)
            y = self.model.generate(x, 64)[0]
            print("".join([dataset.itos[int(i)] for i in y]))

    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.parameters(), lr=self.lr)
        sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.999, last_epoch=-1)
        return [optim], [sched]

model = Model(len(dataset.vocab))
trainer = pl.Trainer(gradient_clip_val=1.0, max_epochs=25, enable_progress_bar=True, accelerator="gpu" if device == "cuda" else "cpu")
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params
---------------------------------
0 | model | MinGPT | 1.4 M 
---------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.489     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 49.10it/s]And now -k
B&Y,upRo&tjRQzvYFk
R.
AEVpAz'KCi3vLQqKCrVUDya.VGgAl yIhduObgH
Epoch 0: 100%|██████████| 111/111 [00:13<00:00,  8.40it/s, loss=2.35, v_num=91]And now the thoome ate spovele.

INCl or westhiseth th lou,s aif ard ant
Epoch 1: 100%|██████████| 111/111 [00:13<00:00,  8.32it/s, loss=2.05, v_num=91, test_loss=2.280]And now of grent hemin.

DUp O worlouth bring grer gett thines.


FIN IU
Epoch 2: 100%|██████████| 111/111 [00:13<00:00,  8.48it/s, loss=1.87, v_num=91, test_loss=2.120]And now it of theer.
How of worthout, there on him whrat his thert.
Whow
Epoch 3: 100%|██████████| 111/111 [00:12<00:00,  8.57it/s, loss=1.79, v_num=91, test_loss=1.930]And now meer night, I woulds.

MISICINIUS:
Weted I shall with an aperomo
Epoch 4: 100%|██████████| 111/111 [00:12<00:00,  8.62it/s, loss=1.72, v_num=91, test_loss=1.830]And now that to fortus, should towe seecure?
Are thou lost mant sigh mea
Epoch 5: 100%|██████████

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 111/111 [00:15<00:00,  7.08it/s, loss=1.43, v_num=91, test_loss=1.430]


In [7]:
y = model.to(device).model.generate(torch.tensor([dataset.stoi[s] for s in "O God, O God!"]).unsqueeze(0).to(device), 650)
print("".join([dataset.itos[int(i)] for i in y[0]]))

O God, O God!

DORSET:
Why, that, all so? I have thy shall the sent
In my storm an the dream'd, but in the do die:
I comes the hold something, swear that a my gods of
then and him--iss they she shall be burden again.

COMINIUS:
Methou shoulder me was murderth,
As he's to saying
Underns to his cheek'd thee,
Which if the false where't.

LADY CALUS:
A mour good content, is doubt all this silver,
Heless!

ROMEO:
Sir? how comes your perceit that.

CAMILL:
Say I accome,
So he shall name towars! to your soul
teather,
At they chery hading.

MENE:
Master you haste is a thou him, there I can and
may the dry shown anch, as if the dead shrends:
Which is, she so look 
